In [7]:
data = pd.read_csv('../physionet/features.csv', header=None)

In [8]:
data

,0,1,2,3,4,5,6,7,8,9,...,256,257,258,259,260,261,262,263,264,265
0,5.0008,4.4831,5.5323,3.8669,4.7546,4.3804,4.7938,7.2438,8.6037,12.3070,...,0.009362,0.010623,0.010047,0.009063,0.009036,0.023540,0.009719,0.009857,0,74.0
1,8.7023,11.2710,9.5483,8.8666,7.2083,9.5014,7.0456,7.9092,9.7063,15.8070,...,0.014644,0.013884,0.015048,0.013964,0.013928,0.013884,0.013884,0.014000,1,49.0
2,40.7450,17.9690,35.6400,25.9630,37.2010,19.5340,11.9040,34.0360,29.9810,39.0550,...,0.089137,0.087087,0.086163,0.089050,0.089630,0.088998,0.087694,0.090561,1,81.0
3,33.1500,22.6410,29.3600,24.2920,29.2000,20.3100,22.3840,16.2330,33.5610,53.9110,...,0.155450,0.525240,0.155330,0.156290,0.155500,0.154930,0.155170,0.154390,0,45.0
4,4.7942,11.3570,8.4035,7.6351,3.7995,9.7095,4.8837,6.3525,8.9160,15.7630,...,0.048165,0.043946,0.047811,0.050805,0.041507,0.153140,0.153140,0.038699,0,53.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6872,12.9620,14.9570,9.1888,13.2170,8.3736,10.5610,5.0672,24.7870,29.4760,39.6600,...,0.041613,0.053335,0.039511,0.041807,0.041072,0.041072,0.041072,0.041146,0,80.0
6873,4.1507,3.4149,4.0208,3.2145,3.6899,3.0816,1.3887,7.2137,11.3060,11.9040,...,0.020881,0.021236,0.026159,0.021079,0.021618,0.021541,0.020607,0.020607,1,62.0
6874,9.5840,8.1037,12.8470,6.1293,10.5870,9.6117,12.1770,18.9970,13.4160,9.8363,...,0.005547,0.005574,0.005322,0.030342,0.005704,0.006375,0.006012,0.012676,0,78.0
6875,9.3122,5.6253,10.4080,5.6657,9.4305,6.9450,6.6328,22.0330,31.0590,26.5710,...,0.140220,0.138420,0.139320,0.140810,0.140160,0.138760,0.138030,0.137700,1,-1.0


In [9]:
mean = np.mean(data)

In [11]:
std = np.std(data)

In [12]:
np.save('mean.npy', mean)
np.save('std.npy', std)

In [4]:
import get_12ECG_features
import numpy as np
import os
import sys
from scipy.io import loadmat
from run_12ECG_classifier import load_12ECG_model, run_12ECG_classifier
import pandas as pd

In [5]:
def load_challenge_data(filename):

    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)

    new_file = filename.replace('.mat', '.hea')
    input_header_file = os.path.join(new_file)

    with open(input_header_file, 'r') as f:
        header_data = f.readlines()

    return data, header_data

def save_challenge_predictions(output_directory, filename, scores, labels, classes):

    recording = os.path.splitext(filename)[0]
    new_file = filename.replace('.mat', '.csv')
    output_file = os.path.join(output_directory, new_file)

    labels=np.asarray(labels,dtype=np.int)
    scores=np.asarray(scores,dtype=np.float64)

    # Include the filename as the recording number
    recording_string = '#{}'.format(recording)
    class_string = ','.join(classes)
    label_string = ','.join(str(i) for i in labels)
    score_string = ','.join(str(i) for i in scores)

    with open(output_file, 'w') as f:
        f.write(recording_string + '\n' + class_string + '\n' + label_string + '\n' + score_string + '\n')


# Find unique number of classes
def get_classes(input_directory, files):

    classes = set()
    for f in files:
        g = f.replace('.mat', '.hea')
        input_file = os.path.join(input_directory, g)
        with open(input_file, 'r') as f:
            for lines in f:
                if lines.startswith('#Dx'):
                    tmp = lines.split(': ')[1].split(',')
                    for c in tmp:
                        classes.add(c.strip())

    return sorted(classes)

In [23]:
input_directory = 'input/PhysioNetChallenge2020_Training_CPSC/test'
output_directory = 'output/'

# Find files.
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)

if not os.path.isdir(output_directory):
    os.mkdir(output_directory)

classes = get_classes(input_directory, input_files)

# Iterate over files.
print('Extracting 12ECG features...')
num_files = len(input_files)
num_features = 182

features = np.zeros((num_files, num_features))

for i, f in enumerate(sorted(input_files)):
    if i%20 == 0:
        print('    {}/{}...'.format(i+1, num_files))

    tmp_input_file = os.path.join(input_directory, f)
    data, header_data = load_challenge_data(tmp_input_file)
    
    tempFeatures = get_12ECG_features.get_12ECG_features(data, header_data)
    features[i, :] = tempFeatures['features']

print('Done.')

Extracting 12ECG features...
    1/96...
    21/96...
    41/96...
    61/96...
    81/96...
Done.


In [25]:
features.shape

(96, 182)